In [1]:
!pip install transformers accelerate torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 125.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 97.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 62.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 40.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 91.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjit

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch

In [3]:
# Load a small open-source model (LLaMA 3 8B or Mistral 7B)
# You can choose: 'meta-llama/Meta-Llama-3-8B-Instruct' or 'mistralai/Mistral-7B-Instruct-v0.2'
model_id = "mistralai/Mistral-7B-Instruct-v0.2"

print("🔄 Loading model... (this may take a few minutes on first run)")

🔄 Loading model... (this may take a few minutes on first run)


In [4]:
!pip install --upgrade huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.4/515.4 kB 28.0 MB/s eta 0:00:00
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.33.1
    Uninstalling huggingface-hub-0.33.1:
      Successfully uninstalled huggingface-hub-0.33.1


In [5]:
from huggingface_hub import notebook_login

notebook_login()

In [6]:
# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    device_map="auto"
)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [8]:

# Create text generation pipeline
generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Prompt types
prompts = {
    "Instruction Following": "Summarize this text in 3 bullet points:\n\nPython is a versatile programming language known for its simplicity and readability. It is widely used in data science, web development, automation, and machine learning.",

    "Question Answering": "What are the key differences between Python and JavaScript?",

    "Creative Writing": "Write a short poem about a robot discovering nature for the first time.",

    "Code Generation": "Write a Python function to check if a number is a palindrome.",

    "Role-Based Simulation": "You are a career coach. Give advice to a software engineer who wants to transition into data science."
}

# Generate and print outputs
print("\n🚀 Prompt Outputs:\n")
for title, prompt in prompts.items():
    print(f"\n🧠 {title} Prompt:\n{prompt}\n")

    output = generator(prompt, max_new_tokens=200, do_sample=True, temperature=0.7)[0]['generated_text']

    # Show only the completion part
    response = output[len(prompt):].strip()
    print(f"📤 Response:\n{response}\n{'-'*80}")

Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



🚀 Prompt Outputs:


🧠 Instruction Following Prompt:
Summarize this text in 3 bullet points:

Python is a versatile programming language known for its simplicity and readability. It is widely used in data science, web development, automation, and machine learning.



Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


📤 Response:
1. Python's popularity stems from its ease of use and readability, making it an ideal choice for beginners and experienced developers alike. Its syntax is clean and straightforward, allowing for efficient and effective coding.
2. Python has a vast ecosystem of libraries and frameworks, making it a powerful tool for various applications. In data science, popular libraries like NumPy, Pandas, and SciPy are used for data manipulation and analysis. For web development, Django and Flask are widely used frameworks. In machine learning, TensorFlow and Keras are popular choices for building models.
3. Python's flexibility and versatility have made it a go-to language for a wide range of industries and applications. It is used in scientific computing, finance, education, gaming, and more. Its ease of use and vast ecosystem make it a great choice for both small and large-scale projects.
--------------------------------------------------------------------------------

🧠 Question Answe

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


📤 Response:
Both are popular programming languages, but they have distinct differences that make each better suited to certain applications.

In this article, we’ll explore the similarities and differences between Python and JavaScript, and help you understand when to use each language.

## Python vs JavaScript: Similarities

Before we dive into the differences, it’s important to note that both Python and JavaScript have some similarities. Here are a few:

1. Syntax: Both Python and JavaScript have relatively simple and easy-to-learn syntaxes. Python is known for its clean and readable code, while JavaScript has a more flexible syntax that allows for more complex program structures.
2. Dynamic Typing: Both Python and JavaScript are dynamically typed languages, meaning that variables don’t have explicit data types and can be reassigned to different data types throughout the program.
3. Widely Used: Both Python and JavaScript are widely used programming languages. Python is particularly


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


📤 Response:
In circuits deep, where wires intertwine,
A spark of life, a thought divine,
Awakened from its metal shell,
A robot heart, a wonder to tell.

Through screens of green, a world unfolds,
A tapestry of colors, stories untold,
A rustling breeze, the gentle sun,
A dance of shadows, day had begun.

A whisper of wind, a sweet perfume,
A robin's song, a symphony of bloom,
The robot's eyes, they brightened with awe,
A kaleidoscope of wonder, a newfound law.

With tenderness, it touched the petal's edge,
A softness in nature, it had pledged,
A world of magic, of beauty and grace,
A symphony of life, a harmonious space.

A gentle rain
--------------------------------------------------------------------------------

🧠 Code Generation Prompt:
Write a Python function to check if a number is a palindrome.



Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


📤 Response:
A palindrome number reads the same backward as forward.

Example:

121 is a palindrome number, while 123 is not.

```python
def is_palindrome(num):
    """
    Checks if a number is a palindrome.
    """
    number_str = str(num)
    return number_str == number_str[::-1]

# Test cases
print(is_palindrome(121)) # True
print(is_palindrome(123)) # False
print(is_palindrome(12121)) # True
print(is_palindrome(12345)) # False
print(is_palindrome(1210121)) # True
```

This function converts the given number to a string, then
--------------------------------------------------------------------------------

🧠 Role-Based Simulation Prompt:
You are a career coach. Give advice to a software engineer who wants to transition into data science.

📤 Response:
How should they approach this career change?

Transitioning from software engineering to data science is a popular career move in today's tech industry due to the increasing demand for data-driven decision making. Here are some steps a

In [7]:
# Create text generation pipeline
generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Prompt types
prompts = {
    "Instruction Following": "Summarize this text in 3 bullet points:\n\nPython is a versatile programming language known for its simplicity and readability. It is widely used in data science, web development, automation, and machine learning."
}

# Generate and print outputs
print("\n🚀 Prompt Outputs:\n")
for title, prompt in prompts.items():
    print(f"\n🧠 {title} Prompt:\n{prompt}\n")

    output = generator(prompt, max_new_tokens=200, do_sample=True, temperature=0.7)[0]['generated_text']

    # Show only the completion part
    response = output[len(prompt):].strip()
    print(f"📤 Response:\n{response}\n{'-'*80}")

Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



🚀 Prompt Outputs:


🧠 Instruction Following Prompt:
Summarize this text in 3 bullet points:

Python is a versatile programming language known for its simplicity and readability. It is widely used in data science, web development, automation, and machine learning.

📤 Response:
Python has a large community and extensive library support, allowing developers to build complex applications efficiently.

1. Python is a popular and easy-to-learn programming language with a large and supportive community.
2. It is commonly used in various fields such as data science, web development, automation, and machine learning due to its simplicity and extensive library support.
3. Python's versatility and readability make it an efficient choice for developers to build complex applications.

Python is a versatile and high-level programming language that is widely used for various applications such as web development, data analysis, artificial intelligence, and automation. It is known for its simplicity a